In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def generate_pulse(n):
    return np.random.choice([-1, 1], size=n)

def generate_noise(n, N0):
    segma = np.sqrt(N0 / 2)
    return np.random.normal(loc=0, scale=segma, size=n)

def transmit_over_channel(n, N0):
    source_bits = generate_pulse(n)
    noise = generate_noise(n, N0)
    return source_bits + noise

def generate_filter(A, type):
    if type == "ramp":
        return np.arange(0, A, 0.1)
    return np.array([A]) # matched filter or delta

def convolve(signal, filter):
    return np.convolve(signal, filter, mode='same')

def sample_and_decode(signal, T):
    samples = signal[::T]
    return np.where(samples >= 0, 1, 0)

n = 1000
N0 = 0.5
T = 1

received_signal = transmit_over_channel(n, N0)
